In [3]:
import requests
import json
import networkx as nx
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer

from torch.nn.functional import cosine_similarity

-------------------------

In [89]:
# Récupération des données de l'API HAL
url = "https://api.archives-ouvertes.fr/search/"
params = {
    'q': 'labStructAcronym_s:"LIRIS"',  # Recherche par acronyme de laboratoire
    'fl': 'title_s,abstract_s,authFullName_s,producedDateY_i,keyword_s', # Champs à récupérer
    'rows': 10000  # Nombre de résultats à récupérer 10000 pour avoir tous les résultats
}
# Réponse de l'API
reponse = requests.get(url, params=params)
data = reponse.json()

# Récupération des documents
documents = data['response']['docs']

# Remplacement des valeurs manquantes par des chaînes vides
categories = ['title_s', 'abstract_s', 'authFullName_s', 'keyword_s']
for doc in documents:
    for cat in categories:
        if cat not in doc:
            doc[cat] = ['']

# Ecriture des documents dans un json
with open('documents.json', 'w', encoding='utf-8') as f:
    json.dump(documents, f, ensure_ascii=False, indent=4)


In [ ]:
# Chargement du modèle
model = SentenceTransformer('all-MiniLM-L6-v2')

# Extraction des informations des documents
informations = [
    f"{doc.get('title_s', '')} "  # Titre
    f"{', '.join(doc.get('keyword_s', []))} "  # Mots-clés
    f"{doc.get('abstract_s', '')} "  # Abstract
    f"Auteurs: {', '.join(doc.get('authFullName_s', []))} "  # Auteurs
    for doc in documents
]

# Générer les embeddings
embeddings = model.encode(informations, convert_to_tensor=True)

# Associer chaque embedding à son document
documents_valides = [doc for doc, titre in zip(documents, informations) if titre.strip()]
documents_embeddings = [
    {"document": doc, "embedding": embedding}
    for doc, embedding in zip(documents_valides, embeddings)
]

In [91]:
def recherche_semantique(query, embeddings, documents, model):
    # Générer l'embedding pour la requête utilisateur
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Calculer la similarité cosinus entre la requête et chaque document
    similarites = cosine_similarity(query_embedding, embeddings)

    # Trouver les indices des documents les plus pertinents
    indices_tries = similarites.argsort(descending=True)

    # Retourner les documents triés par pertinence
    resultats = [documents[i] for i in indices_tries[:5]]
    return resultats

# Exemple de requête
query = "Cedric chauve"
resultats = recherche_semantique(query, embeddings, documents_valides, model)

# Afficher les résultats
for res in resultats:
    print(f"Titre: {res['title_s']}, Mot-clés: {res['keyword_s']}), Auteurs: {res['authFullName_s']}")

In [ ]:
# Chercher si un auteur a publié plusieurs fois (pour mieux visualiser dans le graphe après)
authors = {}
for doc in data['response']['docs']:
    for author in doc['authFullName_s']:
        if author in authors:
            authors[author] += 1
        else:
            authors[author] = 1
            
for author, count in authors.items():
    if count > 1:
        print(f"{author} a publié {count} fois")

In [ ]:
G2 = nx.Graph()
# Création des noeuds et des arêtes
for pub in data['response']['docs']:
    # Création du noeud article
    title = pub.get('title_s')[0]
    year = pub['producedDateY_i']
    article_node = f"Article: {title} ({year})"
    G2.add_node(article_node, type='article', year=year)
    
    # Création des noeuds auteurs et des arêtes qui relient les auteurs à l'article
    authors = pub.get('authFullName_s', [])
    for author in authors:
        author_node = f"Auteur: {author}"
        G2.add_node(author_node, type='auteur')
        G2.add_edge(article_node, author_node, relation='WRITTEN_BY')
    
    # Création des noeuds mots-clés et des arêtes qui relient les mots-clés à l'article
    keywords = pub.get('keyword_s', [])
    for keyword in keywords:
        keyword_node = f"Mot-clé: {keyword}"
        G2.add_node(keyword_node, type='mot-clé')
        G2.add_edge(article_node, keyword_node, relation='HAS_KEYWORD')
        

In [ ]:
print('Nombre de noeuds:', G2.number_of_nodes())

In [ ]:
# Visualisation du graphe
plt.figure(figsize=(20, 20))
pos = nx.spring_layout(G2, k=0.3)
node_colors = ["blue" if G2.nodes[n]["type"] == "article" 
                   else "red" if G2.nodes[n]["type"] == "auteur" 
                   else "orange" for n in G2.nodes()]
nx.draw(G2, pos, node_size=20, with_labels=False, node_color=node_colors)

# Récupération des données du LIRIS

In [2]:
from bs4 import BeautifulSoup
import os
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException

In [6]:
def fetch_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        return BeautifulSoup(response.text, "html.parser")
    else:
        print(f"Erreur {response.status_code} pour {url}")
        return None

In [7]:
def update_json(dataG, section, new_data):
    if os.path.exists("equipes.json"):
        # Lire les données existantes
        with open("equipes.json", 'r', encoding='utf-8') as fichier:
            try:
                contenu = json.load(fichier)
            except json.JSONDecodeError:
                contenu = {}  # Initialiser comme une liste vide si le fichier est vide ou corrompu
    else:
        contenu = {}  # Si le fichier n'existe pas, initialiser une liste vide

    # Ajouter les nouvelles données
    dataG[section] = new_data
    contenu.update(dataG)

    with open("equipes.json", 'w', encoding='utf-8') as fichier:
        json.dump(contenu, fichier, indent=4, ensure_ascii=False)

In [8]:
page = fetch_page("https://liris.cnrs.fr/recherche/liste-des-equipes")

blocks = page.find_all("div", "views-row")

data= {}
equipes = {}
poles = {}

for block in blocks:
    url = block.find("a")
    urlEquipe = "https://liris.cnrs.fr/" + url['href']
    acronyme = url.text

    nomEquipe = block.find("i")

    polePrincipal = block.find("div", "views-field views-field-field-equipel-pole").find("a")
    urlPoleP = "https://liris.cnrs.fr/" + polePrincipal['href']
    nomPoleP = polePrincipal.text

    equipes[acronyme] = {
        "url": urlEquipe,
        "nom": nomEquipe.text,
        "poles": [
            {
                "nom": nomPoleP,
                "type": "Principal"
            }
        ]
    }

    poles[nomPoleP] ={
        "url": urlPoleP,
        "nom": nomPoleP
    }

    poleSecondaires = block.find("span", "views-field views-field-field-poles-secondaires")
    if poleSecondaires:
        poleSecondaires = poleSecondaires.find("span", "field-content").find_all("a")

        for poleSecondaire in poleSecondaires:
            urlPoleS = poleSecondaire['href']
            nomPoleS = poleSecondaire.text
            equipes[acronyme]["poles"].append({
                "nom": nomPoleS,
                "type": "Secondaire"
            })
            poles[nomPoleS] = {
                "url": urlPoleS,
                "nom": nomPoleS
            }

update_json(data, "equipes", equipes)

In [9]:
# Description des pôles
for pole in poles.keys():
    page = fetch_page(poles[pole]['url'])
    if page is not None:
        description = page.find("div", "field-items").text
        poles[pole]["description"] = description

update_json(data, "poles", poles)

In [42]:
# Responsables des équipes
for equipe in equipes.keys():
    page = fetch_page(equipes[equipe]['url'])
    if page is not None:
        responsables = page.find_all("td")
        responsable = responsables[0].text
        responsableAdjoint = responsables[1].text
        equipes[equipe]["responsable"] = responsable.split(":")[1].strip()
        equipes[equipe]["responsableAdjoint"] = responsableAdjoint.split(":")[1].strip()
        

update_json(data, "equipes", equipes)

In [ ]:
# Membres des équipes et description
for equipe in equipes.keys():
    option = Options()
    driver = webdriver.Firefox(options=option)
    driver.get(equipes[equipe]['url'])
    try:
        WebDriverWait(driver, 50).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ev-liris-header-recherche"))
        )
        driver.execute_script("window.scrollTo(0, 200)")
        
        description = driver.find_element("xpath", "//div[@id='qt-affichage_equipe-ui-tabs1']")
        equipes[equipe]["description"] = description.text
        
        # Aller à l'onglet "Membres" et afficher tous les membres
        driver.find_element("xpath", "//a[@href='#qt-affichage_equipe-ui-tabs2']").click()
        driver.find_element("xpath", "//select[@id='edit-items-per-page']").click()
        driver.find_element("xpath", "//option[@value='All']").click()
        membresDict = []
        # While true car la référence de l'élément peu changer
        while True:
            try:
                membres = driver.find_elements("xpath", "//table[@class='views-table cols-5 table table-hover table-striped']/tbody/tr")
                for membre in membres:
                    if membre.find_element("xpath", "./td[1]").text == "":
                        continue
                    prenom = membre.find_element("xpath", "./td[1]").text
                    nom = membre.find_element("xpath", "./td[2]").text
                    statut = membre.find_element("xpath", "./td[3]").text
                    employeur = membre.find_element("xpath", "./td[4]").text
                    
                    membresDict.append({
                        "prenom": prenom,
                        "nom": nom,
                        "statut": statut,
                        "employeur": employeur
                    })
                    
                    
                break  # Si tout s'est bien passé, sortir de la boucle
            except StaleElementReferenceException:
                print("Élément obsolète, relocalisation...")
                continue  # Relocaliser les éléments
        
        equipes[equipe]["membres"] = membresDict
    except TimeoutException:
        print("Chargement de la page trop long")
        continue
    driver.quit()
    
update_json(data, "equipes", equipes)



In [53]:
import networkx as nx

def build_knowledge_graph(equipes):
    G = nx.Graph()

    # Ajouter les équipes et leurs relations
    for equipe in equipes.values():
        # Nœud de l'équipe
        G.add_node(equipe["nom"], type="Équipe", description=equipe["description"], website=equipe["url"])

        # Relation avec les pôles
        for pole in equipe["poles"]:
            G.add_node(pole["nom"], type="Pôle")
            # Relation avec une étiquette Principal/Secondaire
            if pole["type"] == "Principal":
                G.add_edge(equipe["nom"], pole["nom"], relation="BELONGS_TO_PRINCIPAL")
            elif pole["type"] == "Secondaire":
                G.add_edge(equipe["nom"], pole["nom"], relation="BELONGS_TO_SECONDARY")

        # Relation avec le responsable et le co-responsable
        G.add_node(equipe["responsable"], type="Personne", role=equipe["responsable"])
        G.add_edge(equipe["nom"], equipe["responsable"], relation="HAS_RESPONSIBLE")

        G.add_node(equipe["responsableAdjoint"], type="Personne", role=equipe["responsableAdjoint"])
        G.add_edge(equipe["nom"], equipe["responsableAdjoint"], relation="HAS_CO_RESPONSIBLE")

       
        # Ajouter les membres
        for member in equipe["membres"]:
            G.add_node(member["nom"], type="Membre", status=member["statut"], institute=member["employeur"])
            G.add_edge(member["nom"], equipe["nom"], relation="MEMBER_OF")

    return G

knowledge_graph = build_knowledge_graph(equipes)


In [ ]:
# Dessiner le graphe
pos = nx.spring_layout(knowledge_graph)
plt.figure(figsize=(15, 10))

# Dessiner les nœuds
nx.draw_networkx_nodes(knowledge_graph, pos, node_size=5, node_color="lightblue")

# Dessiner les relations
nx.draw_networkx_edges(knowledge_graph, pos, edgelist=knowledge_graph.edges(), edge_color="gray")

# Ajouter les étiquettes
# node_labels = nx.get_node_attributes(knowledge_graph, "type")
# nx.draw_networkx_labels(knowledge_graph, pos, labels={node: node for node in knowledge_graph.nodes()})

# edge_labels = nx.get_edge_attributes(knowledge_graph, "relation")
# nx.draw_networkx_edge_labels(knowledge_graph, pos, edge_labels=edge_labels)

plt.title("Graphe de connaissances")
plt.show()